In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(r'/kaggle/input/ccdata/CC GENERAL.csv')

In [ ]:
df.head()

In [ ]:
df.drop('CUST_ID',axis=1,inplace=True)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df['MINIMUM_PAYMENTS'].describe()

In [ ]:
df['MINIMUM_PAYMENTS'].fillna(df['MINIMUM_PAYMENTS'].mean(),inplace=True)

In [ ]:
df['CREDIT_LIMIT'].describe()

In [ ]:
df['CREDIT_LIMIT'].fillna(df['CREDIT_LIMIT'].mean(),inplace=True)

In [ ]:
df.isnull().sum()

**Check for correlation using heatmap**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
corr=df.corr()
top_features=corr.index
plt.figure(figsize=(20,20))
sns.heatmap(df[top_features].corr(),annot=True)

**Removing highly correlated features(correlation>=0.8)**

In [ ]:
df.drop(['PURCHASES','PURCHASES_FREQUENCY','CASH_ADVANCE_FREQUENCY'],axis=1,inplace=True)

**Outlier Analysis and Removal using Z-score**
* The intuition behind Z-score is to describe any data point by finding their relationship with the Standard Deviation and Mean of the group of data points. Z-score is finding the distribution of data where mean is 0 and standard deviation is 1 i.e. normal distribution.
* While calculating the Z-score we re-scale and center the data and look for data points which are too far from zero. These data points which are way too far from zero will be treated as the outliers. In most of the cases a threshold of 3 or -3 is used i.e if the Z-score value is greater than or less than 3 or -3 respectively, that data point will be identified as outliers

Using box plot to check for outliers

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.boxplot(data=df, width= 0.5,ax=ax,  fliersize=3)


There are a lot of outliers

In [ ]:
df.shape

In [ ]:
from scipy import stats
import numpy as np
z = np.abs(stats.zscore(df))
print(z)

In [ ]:
threshold = 3
print(np.where(z > 3))

In [ ]:
df1 = df[(z < 3).all(axis=1)]

In [ ]:
df1.shape

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.boxplot(data=df1, width= 0.5,ax=ax,  fliersize=3)

**Checking the distribution of data**

In [ ]:
plt.figure(figsize=(20,25), facecolor='white')
plotnumber = 1

for column in df1:
    if plotnumber<=14 :
        ax = plt.subplot(7,2,plotnumber)
        sns.kdeplot(df1[column],bw=1.5)
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.show()

**There is a lot of right skeweness in some of the features**

In [ ]:
df1.head()

**Using Log Transformation to handle skewness**

In [ ]:
df1['log_balance']=np.log(1+df1['BALANCE'])
df1['log_oneoff_purchases']=np.log(1+df1['ONEOFF_PURCHASES'])
df1['log_installments_purchases']=np.log(1+df1['INSTALLMENTS_PURCHASES'])
df1['log_cash_advance']=np.log(1+df1['CASH_ADVANCE'])
df1['log_cash_advance_trx']=np.log(1+df1['CASH_ADVANCE_TRX'])
df1['log_purchases_trx']=np.log(1+df1['PURCHASES_TRX'])
df1['log_credit_limit']=np.log(1+df1['CREDIT_LIMIT'])
df1['log_payments']=np.log(1+df1['PAYMENTS'])
df1['log_minimum_payments']=np.log(1+df1['MINIMUM_PAYMENTS'])

In [ ]:
df1.drop(['BALANCE','ONEOFF_PURCHASES','INSTALLMENTS_PURCHASES','CASH_ADVANCE','CASH_ADVANCE_TRX','PURCHASES_TRX','CREDIT_LIMIT','PAYMENTS','MINIMUM_PAYMENTS'],axis=1,inplace=True)

In [ ]:
df1.head()

In [ ]:
plt.figure(figsize=(20,25), facecolor='white')
plotnumber = 1

for column in df1:
    if plotnumber<=14 :
        ax = plt.subplot(7,2,plotnumber)
        sns.kdeplot(df1[column],bw=1.5)
        plt.xlabel(column,fontsize=20)
    plotnumber+=1
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(df1)
X.shape

# K-Means Clustering
* Elbow curve to determine the K value

In [ ]:
from sklearn.cluster import KMeans
wcss=[]
for i in range (1,12):
    kmeans=KMeans(n_clusters=i,init='k-means++',random_state=40)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,12),wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

**Taking K value as 5**

In [ ]:
kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 50)
y_kmeans = kmeans.fit_predict(X)
print(y_kmeans)

In [ ]:
labels = kmeans.labels_
labels

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(2)
principalComponents = pca.fit_transform(X)
x, y = principalComponents[:, 0], principalComponents[:, 1]
print(principalComponents.shape)

colors = {0: 'red', 1: 'blue', 2: 'green', 3: 'yellow', 4: 'purple'}

In [ ]:
final_df = pd.DataFrame({'x': x, 'y':y, 'label':labels}) 
groups = final_df.groupby(labels)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10)) 

for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=5, color=colors[name], mec='none')
    ax.set_aspect('auto')
    ax.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off')
    ax.tick_params(axis= 'y',which='both',left='off',top='off',labelleft='off')
    
ax.set_title("Customer Segmentation based on Credit Card usage")
plt.show()